In [8]:
# Imports
import tensorflow as tf
import numpy as np

import state
import stateFactory
import nn_utils
from nn_utils import one_hot_encode

In [9]:
quizzes = np.zeros((1000000, 81), np.int32)
solutions = np.zeros((1000000, 81), np.int32)

for i, line in enumerate(open('sudoku.csv', 'r').read().splitlines()[1:]):
    quiz, solution = line.split(",")
    for j, q_s in enumerate(zip(quiz, solution)):
        q, s = q_s
        quizzes[i, j] = q
        solutions[i, j] = s

X = quizzes.reshape((-1, 9, 9))
Y = solutions.reshape((-1, 9, 9))

assert len(X) == len(Y)     # Check same number of X's and Y'x loaded

In [20]:
# Create a new 4D Tensor to hold the One Hot Encoded puzzles
shape = one_hot_encode(X[0]).shape
X_ohe = np.ndarray((len(X), shape[0], shape[1], shape[2]))
Y_ohe = np.ndarray((len(Y), shape[0], shape[1], shape[2]))

In [23]:
delete_ids = []

for i in range(len(X)):
    try:
        X_ohe[i] = one_hot_encode(X[i])
        Y_ohe[i] = one_hot_encode(Y[i])
    except ValueError:
        delete_ids.append(i)
        print(f"skipping {i} due to malformed puzzle")

skipping 14861 due to malformed puzzle
skipping 118801 due to malformed puzzle
skipping 147953 due to malformed puzzle
skipping 168906 due to malformed puzzle
skipping 228197 due to malformed puzzle
skipping 482410 due to malformed puzzle
skipping 760120 due to malformed puzzle
skipping 781414 due to malformed puzzle
skipping 865168 due to malformed puzzle
skipping 994456 due to malformed puzzle


In [2]:
# Create model of the Neural Network

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(9, 9, 9)), # Input layer
    tf.keras.layers.Dense(units=729, activation='relu'), # Hidden layer
    tf.keras.layers.Dense(units=729, activation='relu'), # Hidden layer
    tf.keras.layers.Dense(units=729, activation='relu'), # Output layer
])

In [3]:
# Compiling model

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

In [11]:
# Get data

data = None # TODO: Get data

# Split into test and training data

print(x_train.shape)

(3, 9, 9)


In [14]:
# Fit model to training data
model.fit(x_train, y_train)

NameError: name 'y_train' is not defined

In [16]:
# Evaluate model

model.evaluate(x_test, y_test)



NameError: name 'x_test' is not defined

In [ ]:
# Predict results

test_sudoku = stateFactory.generate_valid_state()
test_sudoku_input = nn_utils.one_hot_encode(test_sudoku.board)

prediction = np.array(model.predict(test_sudoku_input)) # Create 1d-array solution
prediction_arr = prediction.reshape(9, 9, 9) # Put into 3 dimensions
result = nn_utils.get_max_index(prediction_arr) # Extract the max index

print(result)